In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

### Read-in data

In [ ]:
path = '../Data/RAWDATA_Ward_Scenarios.xlsx'
df_wards = pd.read_excel(path)

In [ ]:
df_wards.head()

### Mark wards that change between scenarios

In [ ]:
df_wards['AB_change'] = df_wards.A_color != df_wards.B_color
df_wards['BC_change'] = df_wards.B_color != df_wards.C_color
# df_wards.head()

### Assign 'taxi-rank' numbers to changing wards

In [ ]:
import numpy as np

def _shuffled_range(n):
    """
    Generates a randomly shuffled range from 0 to n-1
    """
    shuffled_range = np.arange(n)
    np.random.shuffle(shuffled_range)
    return shuffled_range

In [ ]:
df_wards['AB_change_no'] = None

change_no = _shuffled_range(len(df_wards[df_wards.AB_change]))
df_wards.loc[df_wards.AB_change, 'AB_change_no'] = change_no

# df_wards[df_wards.AB_change]

In [ ]:
df_wards['BC_change_no'] = None

change_no = _shuffled_range(len(df_wards[df_wards.BC_change])) + len(df_wards[df_wards.AB_change])
df_wards.loc[df_wards.BC_change, 'BC_change_no'] = change_no

# df_wards[df_wards.BC_change]

### View updated dataframe

In [ ]:
df_wards

In [ ]:
df_wards[df_wards['AB_change']]

### Summarise number of beds

In [ ]:
df_wards.groupby(by='A_color').agg(No_Wards=('A_no_beds', 'count'), No_Beds=('A_no_beds', 'sum')).rename_axis(None)

In [ ]:
from cuhvid.wards import get_color_numbers
summ = dict(wards=True, beds=True)
d_A = get_color_numbers(df_wards, 'A', **summ)
d_B = get_color_numbers(df_wards, 'B', **summ)
# get_color_numbers(df_wards, 'A', **summ).join(get_color_numbers(df_wards, 'B', **summ), lsuffix='A', rsuffix='B')

d_A.loc['A','Beds']

# NB: get ordered list of all ward changes + the bed no. deltas 

In [ ]:
from cuhvid.wards2 import Wards

w = Wards()
d = w.get_AB_wards()
dd = d[['id', 'AB_change_no', 'A_color', 'B_color', 'A_no_beds', 'B_no_beds']].sort_values(by='AB_change_no')

# dd = dd.append(pd.Series({'AB_change_no': -1}), ignore_index=True).fillna('').sort_values(by='AB_change_no')

all_colors = ['R', 'A', 'G', 'RG', 'DECANT', 'excl.']

from cuhvid.wards import get_color_numbers
d_A = get_color_numbers(df_wards, 'A', beds=True)

for color in all_colors:
    dd[f'delta_{color}'] = (
        dd.apply(lambda s : -s.A_no_beds if color == s.A_color else 0, axis='columns') 
        + dd.apply(lambda s : s.B_no_beds if color == s.B_color else 0, axis='columns')
    )
    
    
    dd[f'{color}_tot'] = dd[f'delta_{color}'].cumsum() + d_A.loc[color, 'Beds']

# dd.sort_values(by=['delta_R'], ascending=[False])

dd

'''
changeover order hypothesis:
––––––––––––––––––––
| A_color | B_color|
--------------------
| RG      | R      |
| G       | R      |
| G       | A      |
| DECANT  | A      |
| A       | R      |
––––––––––––––––––––
'''

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

all_colors = ['R', 'A', 'G', 'RG', 'DECANT', 'excl.']
line_colors = ['r', 'orange', 'g', 'c', 'k', 'k']
all_totals = [s + '_tot' for s in all_colors]
dd.plot(x='AB_change_no', y=all_totals, color=line_colors, ax=ax)

plt.show()

## __draft: dataframe wrangling

### Melt dataframe to have row for ward in each scenario

In [ ]:
df_melt = df_wards.melt(id_vars=['block', 'id', 'max_no_beds', 'A_no_beds', 'B_no_beds', 'C_no_beds', 'AB_change', 'BC_change'], 
              value_vars=['A_color', 'B_color', 'C_color'], 
              var_name='scenario',
              value_name='color'
             )

### Change scenario column to only contain letter

In [ ]:
df_melt.loc[:,'scenario'] = df_melt.scenario.apply(lambda s: s[0])

### Change three no_beds columns into one column (scenario comes from scenario column now)

#### Create new column

In [ ]:
df_melt['no_beds'] = None
for scenario in df_melt.scenario.unique():
    col = scenario + '_no_beds'
    df_melt.loc[df_melt.scenario == scenario,'no_beds'] = df_melt.loc[df_melt.scenario == scenario,col]

#### Drop old columns

In [ ]:
df_melt = df_melt.drop(columns=['A_no_beds', 'B_no_beds', 'C_no_beds'])

### View melted dataframe

In [ ]:
df_melt.head()

### View all wards that change

In [ ]:
df_melt[df_melt.AB_change | df_melt.BC_change].sort_values(by=['id', 'scenario']).head()

### View all wards for each scenario

In [ ]:
df_melt.pivot(index='id', columns='scenario', values=['color', 'no_beds']).head()